In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from WindPy import w
from sklearn.linear_model import LinearRegression
import datetime
%matplotlib inline

In [2]:
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [3]:
start_date, end_date = '2018-10-22', datetime.datetime.today()

In [5]:
# codes = 'CI005917.WI,CI005919.WI,CI005018.WI,CI005916.WI' # 金融地产、消费、医药、科技
codes = '000934.SH,000931.CSI,000933.SH,000935.SH' # 中证金融地产、可选消费、必选消费、科技

In [6]:
data = w.wsd(codes, 'close', start_date, end_date)

In [7]:
df = pd.DataFrame(np.array(data.Data).T, index=data.Times, columns=data.Codes)

In [8]:
rdf = df.pct_change().dropna()

In [9]:
rdf.tail()

,000934.SH,000931.CSI,000933.SH,000935.SH
2019-10-22,0.000063,0.007347,0.013637,0.017549
2019-10-23,-0.005821,-0.005418,-0.015698,-0.003861
2019-10-24,0.006960,-0.003131,-0.006278,-0.002888
2019-10-25,-0.001875,0.001981,0.011381,0.014107
2019-10-28,0.001059,0.009012,0.013833,0.038209


In [10]:
rdf.iloc[-242:].std() * np.sqrt(242)

000934.SH     0.220999
000931.CSI    0.224177
000933.SH     0.239153
000935.SH     0.317879
dtype: float64

In [11]:
code = '000985.CSI' # 中证全指

In [12]:
data = w.wsd(code, 'close', start_date, end_date)

In [13]:
bdf = pd.DataFrame(np.array(data.Data).T, index=data.Times, columns=data.Codes)

In [14]:
rbdf = bdf.pct_change().dropna()

In [15]:
def objective(coef, args):
    n = coef.size
    X, y = args[0], args[1]
    target = np.dot(X, coef)
#     error = np.sum(np.square(target - y))
    error = np.sum(np.square(target - y)) + np.linalg.norm(coef, 1) # lasso error
    return error / n

def decomposition(code):
    TOLERANCE = 1e-10
    n = rdf.shape[1]
    init_weights = np.array([1. / n] * n)
    X = rdf.values
    y = rbdf[code].values
    
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0},
                   {'type': 'ineq', 'fun': lambda x: x})
    
    opt_res = minimize(fun=objective,
                       x0=init_weights,
                       args=[X, y],
                       constraints=constraints,
                       tol=TOLERANCE,
                       options={'disp': True})
    
    return opt_res.x

In [16]:
weight = decomposition(code)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.250447163975
            Iterations: 14
            Function evaluations: 84
            Gradient evaluations: 14


In [17]:
weight

array([0.30945286, 0.27912244, 0.15742867, 0.25399603])

# 测算风险贡献

In [18]:
covariances = rdf.iloc[-242:].cov()

In [19]:
covariances.to_excel(u'../data/行业收益率协方差.xlsx')

In [20]:
def _allocation_risk(weights, covariances):
    
    # calculate the risk of the weights distribution
    portfolio_risk = np.sqrt((weights * covariances * weights.T))[0, 0]
    
    # It returns the risk of the weights distribution
    return portfolio_risk

In [21]:
def _assets_risk_contribution_to_allocation_risk(weights, covariances):
    
    # calculate the risk of the weights distribution
    portfolio_risk = _allocation_risk(weights, covariances)
    
    # calculate the contribution of each asset to the risk of the weights distribution
    print(covariances * weights.T)
    print(np.multiply(weights.T, covariances * weights.T))
    assets_risk_contribution = np.multiply(weights.T, covariances * weights.T) / portfolio_risk
    
    # It returns the contribution of each asset to the risk of the weights distribution
    return np.abs(assets_risk_contribution)

In [22]:
rc = _assets_risk_contribution_to_allocation_risk(np.matrix(weight), covariances.values)

[[0.00018408]
 [0.00019984]
 [0.00019187]
 [0.00027188]]
[[5.69647954e-05]
 [5.57790626e-05]
 [3.02061217e-05]
 [6.90572958e-05]]


In [23]:
rc

matrix([[0.00391229],
        [0.00383086],
        [0.00207453],
        [0.00474279]])

In [24]:
rc / rc.sum()

matrix([[0.26869264],
        [0.26309976],
        [0.14247682],
        [0.32573078]])

In [25]:
_allocation_risk(np.matrix(weight), covariances.values)

0.014560469618655996

In [26]:
np.matrix(weight) * covariances.values

matrix([[0.00018408, 0.00019984, 0.00019187, 0.00027188]])